In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from numpy.random import choice
from datetime import datetime, date
drive.mount('/content/drive')
base = '/content/drive/My Drive/dati cs/'
file_names = ['GEconV4.csv', 'country_stats.csv', 'lenders.csv', 'loan_coords.csv', 
              'loans.csv', 'loans_lenders.csv', 'locations.csv']

def load():
    def read(name, sep):
      return pd.read_csv(base+name, sep=sep, nrows=None)
    data = {}
    for name in file_names:
        print('Loading {}...'.format(name))
        if name != 'GEconV4.csv':
            data[name] = read(name, ',')
        else:
            data[name] = read(name, ';')
    print('Concluso')
    return data
    
data = load()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Loading GEconV4.csv...
Loading country_stats.csv...
Loading lenders.csv...
Loading loan_coords.csv...
Loading loans.csv...
Loading loans_lenders.csv...
Loading locations.csv...
Concluso


In [2]:
def normalize_loan_lenders(ds):
    def clean_name(x):
        if x[0] == ' ':
            return x[1:]
        else:
            return x
        
    use_db = ds['loans_lenders.csv']
    out = []
    for i in use_db.iterrows():
        lenders = i[1]['lenders']
        id_ = i[1]['loan_id']
        for each in lenders.split(','):
            up = [id_] + [clean_name(each)]
            out.append(up)
    return pd.DataFrame(out, columns=['ID', 'NOME'])

# 1   Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
out_1 = normalize_loan_lenders(data)
normalized_loan_lenders = out_1
display(out_1)

,ID,NOME
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


In [3]:
def make_loan_duration(ds): 
    def make_date(x):
        try:
            x = x.split('.')
            return datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S")
        except:
            return x
        
    use_ks = ['loan_id', 'disburse_time', 'planned_expiration_time']
    use_db = ds['loans.csv'][use_ks]
    
    use_db['disburse_time'] = use_db['disburse_time'].apply(make_date)
    use_db['planned_expiration_time'] = use_db['planned_expiration_time'].apply(make_date)
    diff = use_db['planned_expiration_time'] - use_db['disburse_time']
    ds['loans.csv']['duration'] = diff.dt.total_seconds()/(60*60*24)
    return ds['loans.csv']

# 2    For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.
out_2 = make_loan_duration(data)
# quando 'disburse_time' è nan, anche la differenza è nan: print(out_2[out_2['planned_expiration_time'].isna()]['duration'])
# alcune volte il valore è negativo
display(out_2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53.812569
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96.600775
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37.548669
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34.798634
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57.923634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419602,988180,NaN,NaN,NaN,NaN,400.0,400.0,funded,Tailoring,Services,NaN,KE,Kenya,NaN,shared,0.1,KES,138.0,2015-12-01 04:49:50.000 +0000,2016-01-02 01:00:03.000 +0000,2015-11-23 08:00:00.000 +0000,2015-12-28 15:44:18.000 +0000,14.0,16,4,2,"#Parent, #Repeat Borrower, #Woman Owned Biz",NaN,NaN,monthly,field_partner,39.708368
1419603,988213,Perlita,English,"Perlita is 52 years old, married and has three...","Perlita is 52 years old, married and has three...",300.0,300.0,funded,Pigs,Agriculture,to buy feeds and other supplies to raise her pigs,PH,Philippines,"Numancia, Aklan",shared,0.1,PHP,145.0,2015-12-01 05:13:30.000 +0000,2016-01-02 16:40:07.000 +0000,2015-11-24 08:00:00.000 +0000,2015-12-22 10:37:06.000 +0000,14.0,12,1,1,"#Animals, #Elderly, #Repeat Borrower, #Woman O...",female,true,irregular,field_partner,39.361192
1419604,989109,Okyeso Nyame Group,English,Okyeso Nyame group will begin its third cycle ...,Okyeso Nyame group will begin its third cycle ...,2425.0,2425.0,funded,Bakery,Food,"to buy margarine, flour and sugar at wholesale...",GH,Ghana,Chorkor,shared,0.1,GHS,231.0,2015-12-02 10:56:28.000 +0000,2016-01-03 22:20:04.000 +0000,2015-11-13 08:00:00.000 +0000,2015-12

In [9]:
def lender_atleast_twice(ds):
    use = normalized_loan_lenders
    group = use.groupby('NOME').size().reset_index()
    group.columns = ['NOME','CONTA']
    return group[group['CONTA'] > 1]
    
# 3   Find the lenders that have funded at least twice.
out_3 = lender_atleast_twice(data)
display(out_3)

,NOME,CONTA
0,000,40
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1383792,zzinnia,28
1383793,zzivnk7094,4
1383794,zzmcfate,1730
1383796,zzrvmf8538,15


In [12]:
def n_country_as_borr(ds):
    use_db = ds['loans.csv']
    group = use_db.groupby('country_name').count()['duration']
    return group
    
# 4    For each country, compute how many loans have involved that country as borrowers.
out_4 = n_country_as_borr(data)
display(out_4)

country_name
Afghanistan           2
Albania            3075
Armenia           12809
Azerbaijan         4026
Belize              218
                  ...  
Vietnam           16371
Virgin Islands        0
Yemen              4184
Zambia             1277
Zimbabwe           5075
Name: duration, Length: 96, dtype: int64

In [13]:
def funded_country_as_borrow(ds):
    use_db = ds['loans.csv']
    group = use_db.groupby('country_name').sum()['funded_amount']
    return group

# 5    For each country, compute the overall amount of money borrowed.
out_5 = funded_country_as_borrow(data)
display(out_5)

country_name
Afghanistan        1964650.0
Albania            3992475.0
Armenia           20284675.0
Azerbaijan        14191350.0
Belize              150175.0
                     ...    
Vietnam           23463825.0
Virgin Islands           0.0
Yemen              3340025.0
Zambia             1913250.0
Zimbabwe           5818550.0
Name: funded_amount, Length: 96, dtype: float64

In [14]:
# 6    Like the previous point, but expressed as a percentage of the overall amount lent.
out_6 = 100 * out_5 / sum(out_5)
display(out_6)

country_name
Afghanistan       0.173834
Albania           0.353258
Armenia           1.794810
Azerbaijan        1.255666
Belize            0.013288
                    ...   
Vietnam           2.076104
Virgin Islands    0.000000
Yemen             0.295529
Zambia            0.169286
Zimbabwe          0.514832
Name: funded_amount, Length: 96, dtype: float64

In [15]:
def funded_by_year_by_country(ds):
    use_db = ds['loans.csv']
    use_db['year'] = pd.to_datetime(use_db['disburse_time']).dt.year
    group = use_db.groupby(['country_name', 'year']).sum()['funded_amount']
    return group
    
# 7   Like the three previous points, but split for each year (with respect to disburse time).
out_7 = funded_by_year_by_country(data)
display(out_7)

country_name  year  
Afghanistan   2007.0     194975.0
              2008.0     365375.0
              2009.0     581825.0
              2010.0     563350.0
              2011.0     245125.0
                          ...    
Zimbabwe      2013.0     678525.0
              2014.0    1307975.0
              2015.0     723150.0
              2016.0     787550.0
              2017.0    1214125.0
Name: funded_amount, Length: 748, dtype: float64

In [16]:
def lended_amount_by_lender(ds):
    lends = normalized_loan_lenders
    id_am = ds['loans.csv'][['loan_id', 'funded_amount']]
    inter = pd.merge(lends, id_am, left_on='ID', right_on='loan_id')
    return inter.groupby('NOME').sum()['funded_amount']
    
# 8   For each lender, compute the overall amount of money lent.
out_8 = lended_amount_by_lender(data)
display(out_8)

NOME
000                104125.0
00000               52200.0
0002                33800.0
00mike00             2000.0
0101craign0101     147050.0
                    ...    
zzmcfate          4203225.0
zzpaghetti9994       5000.0
zzrvmf8538          28025.0
zzzsai              21200.0
zzzworld             3000.0
Name: funded_amount, Length: 1383799, dtype: float64

In [17]:
class Exercise_9_routine:
    def __init__(self, db):
        self.use_db         = db['lenders.csv']
        self.loans_db       = db['loans.csv']
        self.loans2lenders  = normalized_loan_lenders
        
    def _2pct(self, x):
        non_na = x[x['country_code'].isnull() == False]
        group  = non_na.groupby('country_code').count()['permanent_name']
        pcts_  = group / group.sum()
        return pcts_
    
    def draw_random_weighted(self, x):
        if pd.isnull(x):
            return choice(list(self.pcts.index), 1, p=self.pcts.tolist())[0]
        else:
            return x
 
    def assume_distribution(self):
        self.pcts = self._2pct(self.use_db)
        sub = self.use_db['country_code'].apply(self.draw_random_weighted)
        self.use_db['country_code'] = sub
        return self.use_db

    def lended_money_by_cc(self):       
        # a
        first_merge = pd.merge(self.loans2lenders, self.use_db, left_on='NOME', right_on='permanent_name')[['ID', 'NOME', 'country_code']]
        # b
        secon_merge = pd.merge(first_merge, self.loans_db[['loan_id', 'funded_amount']], left_on='ID', right_on='loan_id')[['ID', 'NOME', 'country_code', 'funded_amount']]
        # c
        group = secon_merge.groupby('ID').mean()['funded_amount'] / secon_merge.groupby('ID').count()['funded_amount']
        group['ID'] = group.index
        # d
        third_merge = pd.merge(secon_merge, group, left_on='ID', right_on='ID')
        # e 
        group = third_merge.groupby('country_code').sum()
        return group

    
    def run(self):
        '''
        1) calcolo distribuzione di "country_code" e sostituisco nan in "lenders.csv"
        2) per trovare lenders x stati:
            a) incrocio "loans_lenders" con "lenders" e trovo prestatori associati a stati
            b) incrocio (a) con "loans" per trovare quantità prestate a singolo prestito
            c) faccio (b).groupby("ID").mean() / (b).groupby("ID").count()
            d) incrocio (b) con (c) con chiave "ID", assumendo che ogni stato presente 
                nel prestito abbia contribuito equamente
            e) faccio (d).groupby("country_code").sum()
        3) per trovare borrowers x stati faccio "loans".groupby("country_code").sum()
        4) faccio la differenza sulla chiave "country_code" (come indice) tra "lenders" e "borrowers"
        '''
        # 1
        self.use_db = self.assume_distribution()
        # 2
        lended_money_by_cc = self.lended_money_by_cc()
        # 3
        borrowed_money_by_cc = self.loans_db.groupby('country_code').sum()['funded_amount']

        group = pd.merge(lended_money_by_cc, borrowed_money_by_cc, right_index=True, left_index=True)
        lend, borr = group.columns
        return group[lend] - group[borr]

# 9   For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.
out_9 = Exercise_9_routine(data).run()
display(out_9)

country_code
AF     5634175.0
AL    -1981300.0
AM   -18588625.0
AZ   -13604675.0
BA     2995000.0
         ...    
XK    -2399300.0
YE     -791300.0
ZA    35701040.0
ZM      114990.0
ZW    -5153125.0
Length: 94, dtype: float64

In [18]:
def net_lent2pop():
    pops = data['country_stats.csv'].groupby('country_code').sum()['population']
    ratios = out_9 / pops
    ratios = ratios[ratios.notna()]
    high_ratio = {'country':ratios.index[ratios.values.argmax()], 'ratio':ratios.max()}
    return high_ratio

# 10  Which country has the highest ratio between the difference computed at the previous point and the population?
out_10 = net_lent2pop()
display(out_10)

{'country': 'CA', 'ratio': 127.86557939465106}

In [19]:
def net_lent2wealth_pop():
    group = data['country_stats.csv'].groupby('country_code')
    pop = group.sum()['population']
    wealthy_pct = 100 - group.sum()['population_below_poverty_line']
    wealthy_pop = pop * wealthy_pct / 100
    ratios = out_9 / wealthy_pop
    ratios = ratios[ratios.notna()]
    high_ratio = {'country':ratios.index[ratios.values.argmax()], 'ratio':ratios.max()}
    return high_ratio

# 11  Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?
out_11 = net_lent2wealth_pop()
display(out_11)

{'country': 'CA', 'ratio': 141.13198608681134}

In [37]:
class Exercise_12_routine:
    def __init__(self):
        use_ks = ['loan_id', 'country_code', 'disburse_time', 'planned_expiration_time', 'funded_amount']
        self.use_db = data['loans.csv'][use_ks]
        self.use_db['disburse_time'] = pd.to_datetime(self.use_db['disburse_time'])
        self.use_db['planned_expiration_time'] = pd.to_datetime(self.use_db['planned_expiration_time'])

        range_start = min(self.use_db['disburse_time'].min().year, 
                          self.use_db['planned_expiration_time'].min().year)
        range_end   = max(self.use_db['disburse_time'].max().year, 
                          self.use_db['planned_expiration_time'].max().year)

        self.sum_dict = {y:0 for y in range(range_start, range_end + 1)}
        self.sum_dict['date_errors'] = []

    def split_payments(self, start_date, end_date, funding, loan_id):

        def final_date_by_year(year):
            return datetime(year, 12, 31, 23, 59, 59)

        def total_days(start, end):
            return (end - start).total_seconds() / seconds_in_a_day

        def get_years_range(start, end):
            out = [start, final_date_by_year(start.year)]
            while True:
                if out[-1] < end:
                    out.append(final_date_by_year(out[-1].year + 1))
                else:
                    break
            out = out[:-1]
            out.append(end)
            return out

        def date_diff_in_years(start, end):
            return (end - start).total_seconds() / seconds_in_a_day / total_days(start_date, end_date)

        def pro_rate(start, end, fund):
            for single in ranges[1:]:
                date_diff = date_diff_in_years(start, single)

                if date_diff > 0:
                    if len(ranges) > 2:
                        self.sum_dict[single.year] += date_diff * fund
                    else:
                        self.sum_dict[single.year] += fund

                else:
                    self.sum_dict['date_errors'].append(loan_id)
                    break
                start = single

        days_in_a_year = 365
        seconds_in_a_day = 60*60*24

        start_date = start_date.replace(tzinfo=None).to_pydatetime()
        end_date   = end_date.replace(tzinfo=None).to_pydatetime()

        try:
            ranges = get_years_range(start_date, end_date)
            pro_rate(start_date, end_date, funding)
        except:
            self.sum_dict['date_errors'].append(loan_id)

    def run(self):
        for row in self.use_db.iterrows():
            dat = row[1]
            self.split_payments(dat['disburse_time'], dat['planned_expiration_time'], dat['funded_amount'], dat['loan_id'])
        return self.sum_dict

# 12  For each year, compute the total amount of loans. 
#     Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number 
#     of days in each year. 
#     For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount 
#     of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 
#     5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.
out_12 = Exercise_12_routine().run()
display(out_12)

#test verifica correttezza:
#db=data['loans.csv']
#print('a', db['funded_amount'].sum() - db[db['planned_expiration_time'].isnull()]['funded_amount'].sum())
#print('b', sum([out_12[x] for x in out_12 if type(x) == int]))

# 'a' e 'b' devono combaciare


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


{2005: 0,
 2006: 0,
 2007: 0,
 2008: 0,
 2009: 0,
 2010: 0,
 2011: 645001.2263989488,
 2012: 108908129.13788325,
 2013: 121735433.40108259,
 2014: 148071154.51525062,
 2015: 152366832.6849567,
 2016: 150098496.32797647,
 2017: 162660772.81807458,
 2018: 3740204.8883760143,
 'date_errors': [251406,
  252537,
  253044,
  254012,
  251336,
  254221,
  251409,
  253861,
  254067,
  254145,
  254290,
  254413,
  252465,
  252846,
  253073,
  253423,
  253469,
  254260,
  252002,
  251657,
  252624,
  252749,
  253708,
  251710,
  253767,
  253826,
  251307,
  252110,
  252489,
  252683,
  253062,
  253739,
  251361,
  251797,
  252927,
  253163,
  252142,
  252213,
  252270,
  254106,
  252364,
  252976,
  253793,
  252021,
  252088,
  252186,
  253524,
  252991,
  253137,
  253184,
  252837,
  251986,
  252449,
  252939,
  253328,
  253536,
  253313,
  251505,
  252378,
  252989,
  252993,
  253434,
  251567,
  251878,
  254014,
  253251,
  254040,
  251496,
  251868,
  252344,
  252814,
 

In [38]:
def attach_totals_to_lenders():
    # {'country_code' := NaN} è considerato uno stato
    sum_by_country = data['loans.csv'].groupby('country_code', as_index=False).sum()[['country_code', 'funded_amount']]
    to_cross = data['lenders.csv']
    return pd.merge(to_cross, sum_by_country, left_on='country_code', right_on='country_code')

#  13 For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state

out_13 = attach_totals_to_lenders()
display(out_13)

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,funded_amount
0,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0,36365340.0
1,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0,36365340.0
2,andrew86079135,Andrew,NaN,NaN,US,1461301091,NaN,NaN,5.0,Peter Tan,0,36365340.0
3,nguyen6962,Nguyen,NaN,NaN,US,1461301154,NaN,NaN,NaN,NaN,0,36365340.0
4,sirinapa6764,Sirinapa,NaN,NaN,US,1461301496,NaN,NaN,1.0,NaN,0,36365340.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1843987,peter98914650,peter,Juba,Western Bahr el Ghazal,SS,1292061528,business man,to pay my children school fee as you know sout...,NaN,NaN,0,2415650.0
1843988,patrick3790,Patrick,Juba,Central Equatoria State,SS,1295809867,Businessman,I need a laon because I was a contractor and I...,NaN,NaN,0,2415650.0
1843989,martha8370,Martha,NaN,NaN,SS,1307378200,NaN,NaN,2.0,NaN,0,2415650.0
1843990,scott7286,Scott,Mundri,NaN,SS,1326481548,Missionary,NaN,11.0,Chosei,1,2415650.0


In [39]:
def high_avg_occupation():
    names_jobs = data['lenders.csv'][['permanent_name', 'occupation']]
    names_jobs = names_jobs[names_jobs['occupation'].notnull()]
    
    first_merge = pd.merge(out_1, names_jobs, left_on='NOME', right_on='permanent_name')[['ID', 'NOME', 'occupation']]
    secon_merge = pd.merge(first_merge, data['loans.csv'], left_on='ID', right_on='loan_id')[['NOME', 'occupation', 'funded_amount']]

    group = secon_merge.groupby('NOME').sum()
    
    third_merge = pd.merge(first_merge, group, left_on='NOME', right_on='NOME')[['NOME', 'occupation', 'funded_amount']]
    return third_merge.groupby('occupation').mean().idxmax()['funded_amount']

#  14  What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?
out_14 = high_avg_occupation()
display(out_14)

'Faith | Care | Evangelism'

In [40]:
def cluster_loans():
    use_db = data['loans.csv'][['disburse_time', 'funded_amount']]
    use_db['disburse_time'] = pd.to_datetime(use_db['disburse_time'])

    _to_year = lambda x: x.year
    _to_month = lambda x: x.month

    use_db['yearly'] = use_db['disburse_time'].apply(_to_year)
    use_db['monthly'] = use_db['disburse_time'].apply(_to_month)

    group = use_db.groupby(['yearly', 'monthly']).sum()['funded_amount']
    return group

#  15 Cluster the loans according to the year-month of disburse time.
out_15 = cluster_loans()
display(out_15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



yearly  monthly
2005.0  4.0          102850.0
2006.0  3.0            1000.0
        5.0           42325.0
        6.0           33325.0
        7.0           41825.0
                      ...    
2017.0  11.0       12223400.0
        12.0        4257775.0
2018.0  1.0          415250.0
        2.0          327250.0
        3.0           29050.0
Name: funded_amount, Length: 145, dtype: float64

In [41]:
def gni_prod():
    use_db = data['country_stats.csv']
    use = use_db.groupby('country_name').sum()[['population', 'gni']] # trucco per usare 'country_name' come index del DataFrame
    use['gni_prod'] = use['population'] * use['gni']
    return use['gni_prod']

#  16 For each country, compute its overall GNI, by multiplying the per capita GNI with its population
out_16 = gni_prod()
display(out_16)

country_name
Afghanistan    6.647064e+10
Albania        3.004165e+10
Algeria        5.591568e+11
Angola         1.873693e+11
Argentina      9.272623e+11
                   ...     
Venezuela      4.837695e+11
VietNam        5.096650e+11
Yemen          6.498366e+10
Zambia         5.920916e+10
Zimbabwe       2.624491e+10
Name: gni_prod, Length: 174, dtype: float64